# 作业2：对Bert进行微调，完成QA任务

**如果你对Bert没有了解，请先观看视频 [BERT 论文逐段精读【论文精读】](https://www.bilibili.com/video/BV1PL411M7eQ)**

注：本次作业并不需要预先了解任何Transformer的知识，如有兴趣，可以在观看Bert的视频前，先预习 [Transformer论文逐段精读【论文精读】](https://www.bilibili.com/video/BV1pu411o7BE)，后续课程中会讲解Transformer的知识。



In [1]:
!pwd

/Users/jeremy/Desktop/File/一年级下/深度学习/deep-learning-coursework/2 - RNN


In [2]:
import pandas as pd
import json
from tqdm import tqdm
import torch
import numpy as np
import random

device = "cuda" if torch.cuda.is_available() else "cpu"


# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


same_seeds(0)

In [4]:
with open('../input/cmrc2018/train.json') as f:
    train = json.load(f)

with open('../input/cmrc2018/dev.json') as f:
    dev = json.load(f)


Let's have a glance at the data.

In [5]:
train['data'][0]

{'paragraphs': [{'id': 'TRAIN_186',
   'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
   'qas': [{'question': '范廷颂是什么时候被任为主教的？',
     'id': 'TRAIN_186_QUERY_0',
     'answers': [{'text': '1963年', 'answer_start': 30}]}

In [6]:
from transformers import BertTokenizerFast, BertForQuestionAnswering

# You can explore more pretrained models from https://huggingface.co/models
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = BertForQuestionAnswering.from_pretrained('bert-base-chinese').to(device)

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

## PreProcessing

### Prepare training data

In [ ]:
paragraphs = []
questions = []
start_positions = []
end_positions = []
for paragraph in train['data']:
    for qa in paragraph['paragraphs'][0]['qas']:
        
        ### START CODE HERE ### 
        # For each question, add its paragraph, question, start_position and end_position(after calculation) to its corresponding list.
        paragraphs.append(###)
        questions.append(###)
        start_positions.append(###)
        end_positions.append(###)
        ### END CODE HERE ### 

In [8]:
train_encodings = tokenizer(paragraphs, questions, 
                            return_tensors='pt', padding=True, truncation=True,
                           max_length=512)

# `char_to_token` will convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
train_encodings['start_positions'] = torch.tensor([train_encodings.char_to_token(idx, x) if train_encodings.char_to_token(idx, x) != None else -1
                                      for idx, x in enumerate(start_positions)])
train_encodings['end_positions'] = torch.tensor([train_encodings.char_to_token(idx, x-1) if train_encodings.char_to_token(idx, x-1) != None else -1
                                    for idx, x in enumerate(end_positions)])

- 在问答任务中，`input_ids` 是将输入文本转换为整数序列后的输出。它将每个单词或子词映射到一个唯一的整数 ID, 位于 [CLS] 和 [SEP] 标记会被分别映射到一个特殊的 ID，(101: CLS, 102: SEP)。具体可以参考下方例子。

- 在 `token_type_ids` 中，这些标记的值通常为 0 或 1，其中 0 表示该 token 属于第一个文本序列（通常是问题），1 表示该 token 属于第二个文本序列（通常是段落）。

- 在 `attention_mask` 中，0 表示对应的标记应该被忽略，1 表示对应的标记应该被关注。当输入序列长度不足最大长度时，我们需要在序列末尾填充一些无意义的标记，以使序列长度达到最大长度。在这种情况下，`tokenizer`将填充的标记的 attention mask 设置为 0，以告诉模型它们不应该被关注。

In [9]:
train_encodings['input_ids']

tensor([[ 101, 5745, 2455,  ..., 4638, 8043,  102],
        [ 101, 5745, 2455,  ..., 1218, 8043,  102],
        [ 101, 5745, 2455,  ..., 4638, 8043,  102],
        ...,
        [ 101, 7027, 1305,  ...,    0,    0,    0],
        [ 101, 7027, 1305,  ...,    0,    0,    0],
        [ 101,  517, 4263,  ...,    0,    0,    0]])

In [10]:
print(train_encodings['input_ids'].shape)
print(train_encodings['token_type_ids'].shape)
print(train_encodings['attention_mask'].shape)

torch.Size([10142, 512])
torch.Size([10142, 512])
torch.Size([10142, 512])


### Prepare Dataset

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
    
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {k: v[idx].to(device) for k, v in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)

Automatic Mixed Precision (AMP) is available on NVIDIA GPUs that support Tensor Cores, which are specialized hardware units for performing fast matrix multiplication and convolution operations in deep learning. Specifically, Tensor Cores are available on NVIDIA Volta, Turing, and Ampere architectures, which include the following GPU series:

- Volta: Tesla V100, Titan V
- Turing: Quadro RTX, GeForce RTX 20-series, Titan RTX
- Ampere: A100, GeForce RTX 30-series, Titan RTX

In [12]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)
fp16_training = True

if fp16_training:
    !pip install accelerate
    from accelerate import Accelerator

    accelerator = Accelerator()
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [14]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

### START CODE HERE ### 
# Use AdamW as the optimizer, and learning rate 5e-5.
# https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
optim = 
### END CODE HERE ### 


if fp16_training:
    model, optim, train_loader = accelerator.prepare(model, optim, train_loader)
    
model.train()
for epoch in range(3):
    loss_sum = 0.0
    acc_start_sum = 0.0
    acc_end_sum = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for batch_idx, batch in enumerate(pbar):
        optim.zero_grad()
        
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']
        
        outputs = model(input_ids, attention_mask=attention_mask, 
                        start_positions=start_positions, 
                        end_positions=end_positions)
        
        loss = outputs.loss
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        optim.step()
        
        loss_sum += loss.item()
        
        ### START CODE HERE ### 
        # Obtain answer by choosing the most probable start position / end position
        # Using `torch.argmax` and its `dim` parameter to extract preditions for start position and end position.
        start_pred = 
        end_pred = 
        
        # calculate accuracy for start and end positions. eg., using start_pred and start_positions to calculate acc_start.
        acc_start =
        acc_end = 
        ### END CODE HERE ### 
        
        acc_start_sum += acc_start
        acc_end_sum += acc_end
        
        # Update progress bar
        postfix = {
            "loss": f"{loss_sum/(batch_idx+1):.4f}",
            "acc_start": f"{acc_start_sum/(batch_idx+1):.4f}",
            "acc_end": f"{acc_end_sum/(batch_idx+1):.4f}"
        }

        # Add batch accuracy to progress bar
        batch_desc = f"Epoch {epoch}, train loss: {postfix['loss']}"
        pbar.set_postfix_str(f"{batch_desc}, acc start: {postfix['acc_start']}, acc end: {postfix['acc_end']}")


Epoch 2: 100%|██████████| 1268/1268 [04:11<00:00,  5.04it/s, Epoch 2, train loss: 0.8371, acc start: 0.6862, acc end: 0.6900]


In [15]:
def predcit(doc, query):
    print(doc)
    print('提问：', query)
    item = tokenizer([doc, query], max_length=512, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        input_ids = item['input_ids'].to(device).reshape(1,-1)
        attention_mask = item['attention_mask'].to(device).reshape(1,-1)
        
        outputs = model(input_ids[:, :512], attention_mask[:, :512])
        
        ### START CODE HERE ### 
        # Using `torch.argmax` and its `dim` parameter to extract preditions for start position and end position.
        start_pred = 
        end_pred = 
        ### END CODE HERE ### 
    
    try:
        start_pred = item.token_to_chars(0, start_pred)
        end_pred = item.token_to_chars(0, end_pred)
    except:
        return ''
    
    if start_pred.start > end_pred.end:
        return ''
    else:
        return doc[start_pred.start:end_pred.end]

In [17]:
dev['data'][100]

{'paragraphs': [{'id': 'DEV_109',
   'context': '岑朗天（），笔名朗天、霍惊觉。香港作家、影评人、文化活动策划、大学兼职讲师。香港新亚研究所硕士，师从牟宗三，父亲为香港专栏作家昆南。曾在香港多家报社从事繙译、编辑、采访工作。1995年加入香港电影评论学会，并于2003-2007年出任该会会长，2016年退出。1995年参与创立新研哲学会，后易名香港人文哲学会，再易名香港人文学会。1998年加入树宁．现在式单位，出任该剧团董事及编剧。2003年担任牛棚书展（2003-6）统筹，协助开拓主流以外的书展文化（牛棚书展精神后为九龙城书节继承）。2004年6月至2011年加入商业电台光明顶，担任嘉宾主持。2004年至2014年于香港中文大学新闻与传播学院兼职教授媒体创意写作。2012年始兼任香港浸会大学电影学院讲师，教授文学与影视相关课程。',
   'qas': [{'question': '岑朗天笔名叫什么？',
     'id': 'DEV_109_QUERY_0',
     'answers': [{'text': '朗天、霍惊觉', 'answer_start': 8},
      {'text': '朗天、霍惊觉', 'answer_start': 8},
      {'text': '朗天、霍惊觉', 'answer_start': 8}]},
    {'question': '岑朗天的职业都有哪些？',
     'id': 'DEV_109_QUERY_1',
     'answers': [{'text': '作家、影评人、文化活动策划、大学兼职讲师', 'answer_start': 17},
      {'text': '作家、影评人、文化活动策划、大学兼职讲师', 'answer_start': 17},
      {'text': '作家、影评人、文化活动策划、大学兼职讲师', 'answer_start': 17}]},
    {'question': '岑朗天哪年加入香港电影评论学会？',
     'id': 'DEV_109_QUERY_2',
     'answers': [{'text': '1995年', 'answer_start': 87},
      {'te

In [18]:
model.eval()
predcit(dev['data'][100]['paragraphs'][0]['context'],
       dev['data'][100]['paragraphs'][0]['qas'][0]['question'])

岑朗天（），笔名朗天、霍惊觉。香港作家、影评人、文化活动策划、大学兼职讲师。香港新亚研究所硕士，师从牟宗三，父亲为香港专栏作家昆南。曾在香港多家报社从事繙译、编辑、采访工作。1995年加入香港电影评论学会，并于2003-2007年出任该会会长，2016年退出。1995年参与创立新研哲学会，后易名香港人文哲学会，再易名香港人文学会。1998年加入树宁．现在式单位，出任该剧团董事及编剧。2003年担任牛棚书展（2003-6）统筹，协助开拓主流以外的书展文化（牛棚书展精神后为九龙城书节继承）。2004年6月至2011年加入商业电台光明顶，担任嘉宾主持。2004年至2014年于香港中文大学新闻与传播学院兼职教授媒体创意写作。2012年始兼任香港浸会大学电影学院讲师，教授文学与影视相关课程。
提问： 岑朗天笔名叫什么？


'朗天、霍惊觉'

## Open Questions
可以查阅相关资料，并完成如下开放式的问答题。


- 我们使用了512长度的Bert，但是在实际应用中，输入长度可能大于512，你想怎么解决这个问题，请描述你的算法，在训练和预测时分别采取什么样的方法。（假设问题的长度都满足小于512token，段落的长度可能大于512token，以QA问题为例）


Your Answer:

- 在输出中，我们分别对start_pred和end_pred的位置进行预估，如果end_pred<start_pred，我们可以如何解决这样的问题?

Your Answer:

- Bert的分词方式是什么?在中文中，你觉得这样的方式会带来什么问题？什么样的分词方式适合中文？在中文的文本上，除了改变分词方式，还有哪些方式可以提升模型效果？

阅读资料：https://github.com/ymcui/Chinese-BERT-wwm

Your Answer: